# Import

In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraForPreTraining, ElectraTokenizerFast
from torch.optim import AdamW

batch_size = 4

# Wordnet

In [2]:
import nltk
nltk.download('wordnet', '../working')

import zipfile
import os

file_path = os.path.abspath("../working/corpora/wordnet.zip")

with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.dirname(file_path))
    
nltk.data.path.append('../working')

from nltk.corpus import wordnet

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package wordnet to ../working...


# Without any fine tuning

## Create test Dataset and Dataloader

In [3]:
tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-base-discriminator")

class JsonlDataset(Dataset):
    def __init__(self, filename, is_test=False):
        self.data = [json.loads(line) for line in open(filename, 'r', encoding='utf-8')]
        self.is_test = is_test

    def __len__(self):
        return len(self.data)
    
    def getMerged(self, question, options, article):
        wordnet_meaning = []
        for i in range(5):
            synsets = wordnet.synsets(options[i])
            if len(synsets):
                wordnet_meaning.append(f"{synsets[0].definition()}")
            else:
                wordnet_meaning.append("")
        return [
            f"{question.replace('@placeholder', options[i])} [SEP] {article}" for i in range(5)
        ]
    
    def getLabels(self, label):
#         print([(0 if i == label else 1) for i in range(5)])
        return torch.tensor([(0 if i == label else 1) for i in range(5)])
    
    def __getitem__(self, idx):
        item = self.data[idx]
        article = item['article']
        question = item['question']
        options = [item[f'option_{i}'] for i in range(5)]
        
        merged = self.getMerged(question, options, article)
        
        _input = tokenizer(
            merged,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        
        input_ids = _input["input_ids"]
        
        origin = f"{question.replace('@placeholder', '[MASK]')} [SEP] {article}"
        
        origin_ids = tokenizer(
            origin,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )["input_ids"]
        
        option_position = (origin_ids == tokenizer.mask_token_id).nonzero().tolist()[0][1]
        
        if self.is_test:
            return {
                'article': article,
                'question': question,
                'options': options,
                'input_ids': input_ids,
                'merged': merged,
                'option_position': option_position
            }
        else:
            return {
                'article': article,
                'question': question,
                'options': options,
                'label': item['label'],
                'input_ids': input_ids,
                'labels': self.getLabels(item['label']),
                'merged': merged,
                'option_position': option_position
            }

In [4]:
task1_test_dataset = JsonlDataset('../input/semevaldataset/trail_data/Task_1_Imperceptibility.jsonl')
task1_test_loader = DataLoader(task1_test_dataset, batch_size=batch_size, shuffle=True)


task2_test_dataset = JsonlDataset('../input/semevaldataset/trail_data/Task_2_Nonspecificity.jsonl')
task2_test_loader = DataLoader(task2_test_dataset, batch_size=batch_size, shuffle=True)

In [5]:
def eval(model, test_loader):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = model.to(device)
    model.eval()
    
    total = 0
    correct = 0
    for i, batch in enumerate(test_loader):
        with torch.no_grad():
            _input_ids = batch['input_ids'].to(device)
            _labels = batch['labels']
            _label = batch['label']
            _option_position = batch['option_position']
            
            for i in range(_input_ids.shape[0]):
                input_ids = _input_ids[i]
                labels = _labels[i]
                label = _label[i]
                option_position = _option_position[i]
                
                # input_ids shape: torch.Size([5, 512])
                # labels shape: torch.Size([5]
                # print('input ids: ', input_ids.shape, 'labels: ', labels.shape)
            
                outputs = model(input_ids)
                logits = outputs.logits
                
                # logit shape: torch.Size([5, 512])
                # print('logits: ', logits.shape)
                
                # probabilities = torch.sigmoid(logits)
                probabilities = logits[:, option_position]
                # print(probabilities, label)
                
                most_likely_idx = probabilities.argmin().item()
                
                # print(most_likely_idx, label)
                correct += (most_likely_idx == label.item())
                total += 1

    print(correct / total)

## Without any Fine tuning: Task1, Task2

In [6]:
# Evaluation Task1, Cross1, Task2, Cross2
discriminator = ElectraForPreTraining.from_pretrained("google/electra-base-discriminator")

print('Task1: ')
eval(discriminator, task1_test_loader)
print('Task2: ')
eval(discriminator, task2_test_loader)

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForPreTraining: ['electra.embeddings_project.bias', 'electra.embeddings_project.weight']
- This IS expected if you are initializing ElectraForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Task1: 


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


0.822
Task2: 
0.882


# With Wordnet

In [7]:
class JsonlDataset2(JsonlDataset):
    def getMerged(self, question, options, article):
        wordnet_meaning = []
        for i in range(5):
            synsets = wordnet.synsets(options[i])
            if len(synsets):
                wordnet_meaning.append(f"{synsets[0].definition()}")
            else:
                wordnet_meaning.append("")
        return [
            f"{question.replace('@placeholder', options[i])} [SEP] {wordnet_meaning[i]} [SEP] {article}" for i in range(5)
        ]

In [8]:
task1_test_dataset = JsonlDataset2('../input/semevaldataset/trail_data/Task_1_Imperceptibility.jsonl')
task1_test_loader = DataLoader(task1_test_dataset, batch_size=batch_size, shuffle=True)

task2_test_dataset = JsonlDataset2('../input/semevaldataset/trail_data/Task_2_Nonspecificity.jsonl')
task2_test_loader = DataLoader(task2_test_dataset, batch_size=batch_size, shuffle=True)

## Task1, Task2

In [9]:
print('Task1: ')
eval(discriminator, task1_test_loader)
print('Task2: ')
eval(discriminator, task2_test_loader)

Task1: 
0.775
Task2: 
0.857
